In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import websockets
import json
import datetime
from IPython.display import Audio, display

async def send_audio():
    uri = "wss://pools-games-price-bacon.trycloudflare.com/ws"

    async with websockets.connect(
        uri,
        ping_interval=None,
        max_size=10 * 1024 * 1024
    ) as websocket:
        # 1. 메타데이터 전송
        meta = {
            "user_id": "test_user_1",
            "timestamp": datetime.datetime.utcnow().isoformat()
        }
        await websocket.send(json.dumps(meta))
        print("✅ 메타데이터 전송 완료")

        # 2. 오디오 전송
        with open("output.mp3", "rb") as f:
            audio_data = f.read()
        await websocket.send(audio_data)
        print("✅ 오디오 전송 완료")

        # 3. 첫 번째 응답 받기 (JSON or WAV)
        response = await websocket.recv()

        # 바이너리인지 문자열인지 체크
        if isinstance(response, bytes):
            print("📥 바로 오디오 바이너리 수신")
            pcm_data = response
        else:
            try:
                response_json = json.loads(response)
                print("✅ JSON 응답:", response_json)
                # 그 다음 응답은 바이너리 오디오일 것
                pcm_data = await websocket.recv()
            except json.JSONDecodeError:
                print("❌ 예상치 못한 응답 형식:", response)
                return

        # 4. 저장 및 재생
        output_path = "received_from_server.wav"
        with open(output_path, "wb") as f:
            f.write(pcm_data)
        print(f"🎧 저장 완료: {output_path}")

        display(Audio(output_path))

# 실행
await send_audio()
